In [ ]:
#Library


from pandasql import sqldf
import pandas as pd
import numpy as np
import openpyxl
import datetime
from openpyxl import workbook,load_workbook
from sqlalchemy import null
from zmq import NULL
mysql = lambda q: sqldf(q,globals())  	


In [ ]:
#Importing the cancellation dump and the recent TA Portfolio
df1 = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\CANCELLATION DUMP\Cancellation-06092023.xlsx") #CancellationDump Data 
#df1["PromoCode"] = df1["PromoCode"].astype(str)
df1['PromoCode']=df1['PromoCode'].str.upper()
df1['PromoCode']=df1['PromoCode'].str.strip()

df2 = pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\TA PORTFOLIO\TAPortfolio-30042023.xlsx") #TA Portfolio data

df2['Region']=df2['Region'].str.strip()
df2['City']=df2['City'].str.strip()
df2['Agent Id - IATA Id']=df2['Agent Id - IATA Id'].str.strip()
df2['Parent Id']=df2['Parent Id'].str.strip()
df2['NEW Main Channel for Commercial Team']=df2['NEW Main Channel for Commercial Team'].str.strip()

df2 = df2.drop_duplicates(keep='first')

In [ ]:
#Chartered Flight Name
Chatterflight=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChatterFlightMaster.xlsx")

#DOM/INT Master 
domint=pd.read_excel(r"C:\Users\ank2\Desktop\DAILY MIS REPORT\MASTERS\ChannelAndRegionNameMaster.xlsx",sheet_name='INT')


In [ ]:
#Removing this SOC because it's a test id. Needs to be removed
cancel = sqldf("select * from df1 where SourceOrganizationCode not in ('DELS0000','DELI0000','GOMARINE') and PassengerStatus = 'Cancelled' ")

# Copying Values to SourceOrganizationCode where PaymentOrganizationCode is not null for cancellation
coc = sqldf("""Select *,
             CASE WHEN
             SourceOrganizationCode = 'G8' and PaymentOrganizationCode not NULL
             THEN 
             PaymentOrganizationCode
             WHEN
             SourceOrganizationCode = 'INFINITY' and PaymentOrganizationCode not NULL
             THEN 
             PaymentOrganizationCode
             ELSE
             SourceOrganizationCode END AS SOC1
             from cancel""")


In [ ]:
#creating some new variables

#Cancelled Pax
coc['Pax']=np.where(cancel.PassengerStatus == "Cancelled",1,0) 

#Transaction Date :- format needs to be changed in line 56 based on how line 55 is getting read (%Y-%d-%m or %Y-%m-%d)
can_date=pd.to_datetime(coc['CancelledDate']).dt.date
coc['TransactionDate'] = pd.to_datetime(can_date,format="%Y-%d-%m",infer_datetime_format=True).dt.strftime('%d-%m-%Y')


#Departure Date
dep_date=pd.to_datetime(coc['DepartureDate']).dt.date
coc['DepDate'] = pd.to_datetime(dep_date,format="%Y-%m-%d",infer_datetime_format=True).dt.strftime('%d-%m-%Y')

#Booking Date
split_book=coc['BookingDate'].str.split(" ", n = -1, expand = True)

split_book1=split_book[0].str.split("/", n = -1, expand = True)

split_book1["Date"]=split_book1[1]
split_book1["Month"]=split_book1[0]
split_book1["Year"]=split_book1[2]

coc['Booking_Date1']=split_book1['Date'] + "-" + split_book1['Month']+ "-" + split_book1['Year']


split_book2=split_book[0].str.split("-", n = -1, expand = True)

split_book2["Date"]=split_book2[1]
split_book2["Month"]=split_book2[2]
split_book2["Year"]=split_book2[0]

coc['Booking_Date2']=split_book2['Date'] + "-" + split_book2['Month']+ "-" + split_book2['Year']

coc['Booking_Date2']=coc['Booking_Date2'].fillna(coc['Booking_Date1'])
coc['Booking_Date2']=pd.to_datetime(coc['Booking_Date2'],format="%d-%m-%Y",infer_datetime_format=True).dt.strftime('%d-%m-%Y')

#creating some more date related variables
coc['CancelDate'] = coc['TransactionDate'].copy()
coc['DepartureMonth'] = pd.to_datetime(coc['DepDate'],format="%d-%m-%Y",infer_datetime_format=True).dt.strftime('%b-%y')
coc['BookingMonth'] = pd.to_datetime(coc['Booking_Date2'],format="%d-%m-%Y",infer_datetime_format=True).dt.strftime('%b-%y')
coc['cancelMonth'] = pd.to_datetime(coc['CancelDate'],format="%d-%m-%Y",infer_datetime_format=True).dt.strftime('%b-%y')
coc['NOD'] = (pd.to_datetime(coc['DepDate'],format="%d-%m-%Y",infer_datetime_format=True) - pd.to_datetime(coc['CancelDate'],format="%d-%m-%Y",infer_datetime_format=True)) / np.timedelta64(1, 'D')


#testcode for PRV MONTH
#coc['test']=sqldf("""select case when DepDate < "01-09-2022" then 'PRV MONTH' else DepDate end as test from coc""")

#some necessary variables included
coc['sector'] = coc["DepartureStation"] + "-" + coc["ArrivalStation"]
coc['Cancelled_Collected_Revenue']=coc['BaseFare']+coc['ServiceChargeAmount']+coc['OtherTaxes']+coc['FuelTax']
coc['FIT/Grp_Series'] = np.where(coc['FareClass'] != "G","FIT","Grp & Series")
coc['Agent_Role']=''


In [ ]:
coc1 = sqldf("""select * , case when (DepartureStation in (select AirportCode from domint))  then 'INT'
                              when (ArrivalStation in (select AirportCode from domint)) then 'INT'
                   else 'DOM' end as domint from coc""")

#copying values from SourceOrganizationCode to 'Agent Id - IATA Id' in Cancellation dump
coc1['Agent Id - IATA Id'] = coc1['SOC1'].copy()


In [ ]:
#Applying left join on Cancellation dump with TA portfolio
CancellationResult = pd.merge(coc1,df2,on='Agent Id - IATA Id',how = 'left')
#CancellationResult.columns

#Renaming certain columns
CancellationResult.rename(columns={"Agent Id - IATA Id" : "AgentId","Parent Id":"ParentId","ID Type":"IDType","TA Name":"TAName",
                     "Main Channel":"MainChannel","Main Channel \non the basis of\n Parent ID":"MainChannelBasedOnParentID",
                     "Country_Name":"CountryName","Global_Region":"GlobalRegion",
                     "NEW Main Channel for Commercial Team":"NEWMainChannel" },inplace=True)

# CancellationResult['ChannelType']=sqldf("""select case when ChannelType = 'API' then 'DIRECT' else ChannelType end as ChannelType from CancellationResult""")
CancellationResult['ChannelType']=CancellationResult['ChannelType'].str.upper()


In [ ]:
#Replacing the NEWMainChannel with ChannelType if it is NULL or else the value remains intact
canres = sqldf(""" Select *,
              CASE when NEWMainChannel is NULL
              THEN ChannelType
              ELSE NEWMainChannel
              END as NEWMainChannel1
              from CancellationResult
              """)
canres['NEWMainChannel1'] = canres['NEWMainChannel1'].str.upper()


In [ ]:
Chatterflight['combokey']=Chatterflight['FltNo'].astype(str)+Chatterflight['Org']+Chatterflight['Dest']

In [ ]:
#categorizing the rows into chartered and non chartered flights
split=canres["FlightNumber"].str.split("/", n = -1, expand = True)

if(len(split.columns)==1):
    canres['Flight1'] = split[0]
    canres['combokey1'] = canres['Flight1']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight2'] = ''
    canres['combokey2'] = ''
    canres['Flight3'] = ''
    canres['combokey3'] = ''
    canres['Flight4'] = ''
    canres['combokey4'] = ''
elif(len(split.columns)==2):
    canres['Flight1'] = split[0]
    canres['combokey1'] = canres['Flight1']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight2'] = split[1]
    canres['combokey2'] = canres['Flight2']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight3'] = ''
    canres['combokey3'] = ''
    canres['Flight4'] = ''
    canres['combokey4'] = ''
elif(len(split.columns)==3):
    canres['Flight1'] = split[0]
    canres['combokey1'] = canres['Flight1']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight2'] = split[1]
    canres['combokey2'] = canres['Flight2']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight3'] = split[2]
    canres['combokey3'] = canres['Flight3']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight4'] = ''
    canres['combokey4'] = ''
else:
    canres['Flight1'] = split[0]
    canres['combokey1'] = canres['Flight1']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight2'] = split[1]
    canres['combokey2'] = canres['Flight2']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight3'] = split[2]
    canres['combokey3'] = canres['Flight3']+canres['DepartureStation']+canres['ArrivalStation']
    canres['Flight4'] = split[3]
    canres['combokey4'] = canres['Flight4']+canres['DepartureStation']+canres['ArrivalStation']
    
canres1 = sqldf("""select *,case when (combokey1 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey2 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey3 in (Select combokey from Chatterflight)) then 'Charter'
                                                  when (combokey4 in (Select combokey from Chatterflight)) then 'Charter'
                                                  else 'Non-Charter' end as charter_noncharter from canres""")


In [ ]:
#Manually update every month
canres1['Week']=sqldf("""select case when CancelDate between '01-09-2023' and '07-09-2023' then "01-07"
                                     when CancelDate between '08-09-2023' and '14-09-2023' then "08-14"
                                     when CancelDate between '15-09-2023' and '21-09-2023' then "15-21"
                                     when CancelDate between '22-09-2023' and '28-09-2023' then "22-28"
                                     else "29-30" end as Week from canres1""")
                                     
canres1['MTD/FWD']=np.where(canres1['DepartureMonth'] != "Sep-23","FWD","MTD")


In [ ]:
##Manually update the Prev Month Logic
canres1['DepCanDiff'] = (pd.to_datetime(canres1['DepDate'],format="%d-%m-%Y",infer_datetime_format=True) - pd.to_datetime('01-09-2023',format="%d-%m-%Y",infer_datetime_format=True)) / np.timedelta64(1, 'D')
canres1['MTD/FWD']=sqldf("""select case when DepCanDiff < 0 then 'PRV MONTH' else [MTD/FWD] end as [MTD/FWD] from canres1""")

#Changing Region to ChannelType if it's NULL else it remains the same
canres2 = sqldf(""" Select *,
              CASE when Region is NULL
              THEN ChannelType
              ELSE Region
              END as Region1
              from canres1
              """)

canres2['Region1'] = canres2['Region1'].str.upper()


In [ ]:
#filling the values of channel type when the column has values as G8 or Infinity
canres2['TAName']=sqldf(""" select case when TAName is NULL then NEWMainChannel1 else TAName end as TAName from canres2""")

canres2['ParentId']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else ParentId end as ParentId from canres2""")
canres2['ParentId']=canres2['ParentId'].str.upper()

canres2['IDType']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else IDType end as IDType from canres2""")
canres2['IDType']=canres2['IDType'].str.upper()

canres2['City']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else City end as City from canres2""")
canres2['City']=canres2['City'].str.upper()

canres2['TAName']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else TAName end as TAName from canres2""")
canres2['TAName']=canres2['TAName'].str.upper()

canres2['Channel']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else Channel end as Channel from canres2""")
canres2['Channel']=canres2['Channel'].str.upper()

canres2['MainChannel']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else MainChannel end as MainChannel from canres2""")
canres2['MainChannel']=canres2['MainChannel'].str.upper()

canres2['Country_Name ']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else [Country_Name ] end as [Country_Name ] from canres2""")
canres2['Country_Name ']=canres2['Country_Name '].str.upper()

canres2['GlobalRegion']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else GlobalRegion end as GlobalRegion from canres2""")
canres2['GlobalRegion']=canres2['GlobalRegion'].str.upper()

canres2['TAName']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else TAName end as TAName from canres2""")
canres2['TAName']=canres2['TAName'].str.upper()

canres2['Region1']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else Region1 end as Region1 from canres2""")
canres2['Region1']=canres2['Region1'].str.upper()

canres2['MainChannelBasedOnParentID']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else MainChannelBasedOnParentID end as MainChannelBasedOnParentID from canres2""")
canres2['MainChannelBasedOnParentID']=canres2['MainChannelBasedOnParentID'].str.upper()

canres2['NEWMainChannel1']=sqldf("""select case when AgentId = 'G8' or AgentId = 'INFINITY' then ChannelType else NEWMainChannel1 end as NEWMainChannel1 from canres2""")
canres2['NEWMainChannel1']=canres2['NEWMainChannel1'].str.upper()


In [ ]:
#filling the null values as channel type values wrt to the column values.
canres3=sqldf("""select *,
                            case when Region1 is NULL then ChannelType else Region1 end as Region2,                    
                            case when City is NULL then ChannelType else City end as City,
                            case when GlobalRegion is NULL then ChannelType else GlobalRegion end as GlobalRegion,
                            case when Channel is NULL then ChannelType else Channel end as Channel,
                            case when MainChannel is NULL then ChannelType else MainChannel end as MainChannel,
                            case when MainChannelBasedOnParentID is NULL then ChannelType else MainChannelBasedOnParentID end as MainChannelBasedOnParentID,
                            case when [Country_Name ] is NULL then ChannelType else [Country_Name ] end as [Country_Name ],
                            case when TAName is NULL then ChannelType else TAName end as TAName,
                            case when ParentId is NULL then ChannelType else ParentId end as ParentId,
                            case when NEWMainChannel1 is NULL then ChannelType else NEWMainChannel1 end as NEWMainChannel1,
                            case when IDType is NULL then ChannelType else IDType end as IDType                            
                            from canres2""")


In [ ]:
#Replacing API to Direct
canres4 = sqldf("""select *,
                            case when Region2 is 'API' then 'DIRECT' else Region2 end as Region2,
                            case when City is 'API' then 'DIRECT' else City end as City,
                            case when GlobalRegion is 'API' then 'DIRECT' else GlobalRegion end as GlobalRegion,
                            case when Channel is 'API' then 'DIRECT' else Channel end as Channel,
                            case when MainChannel is 'API' then 'DIRECT' else MainChannel end as MainChannel,
                            case when MainChannelBasedOnParentID is 'API' then 'DIRECT' else MainChannelBasedOnParentID end as MainChannelBasedOnParentID,
                            case when [Country_Name ] is 'API' then 'DIRECT' else [Country_Name ] end as [Country_Name ],
                            case when TAName is 'API' then 'DIRECT' else TAName end as TAName,
                            case when ParentId is 'API' then 'DIRECT' else ParentId end as ParentId,
                            case when IDType is 'API' then 'DIRECT' else IDType end as IDType,
                            case when NEWMainChannel1 is 'API' then 'DIRECT' else NEWMainChannel1 end as NEWMainChannel1
                            from canres3""")

In [ ]:
resultcan = sqldf("""select FlightNumber, 
                            DepartureStation,
                            ArrivalStation, 
                            DepartureDate, 
                            BookingDate,
                            CancelledDate, 
                            ChannelType, 
                            FlightStatus,
                            PNR, 
                            PassengerLastNameAndFirstName, 
                            BaseFare,
                            ServiceChargeAmount, 
                            FuelTax, 
                            OtherTaxes, 
                            FareClass, 
                            Cabin, 
                            FareBasis, 
                            PromoCode,
                            PromoCodeAmount,   
                            BookingStatus, 
                            PassengerStatus,
                            PassengerType, 
                            BookingAgentID, 
                            SourceOrganizationCode,
                            SourceOrganizationCodeName, 
                            PaymentOrganizationCode,
                            OrganizationGroup, 
                            PaymentOrganizationName, 
                            Agent_Role as AgentRole,
                            TransactionDate, 
                            DepDate  as  DepartureDate, 
                            Booking_Date2  as  BookingDate, 
                            CancelDate  as  CancelledDate,
                            DepartureMonth  as  Dep_Month, 
                            BookingMonth  as  Booked_Month, 
                            cancelMonth  as  Cancellation_Month, 
                            Cancelled_Collected_Revenue, 
                            AgentId, 
                            ParentId, 
                            TAName,
                            Region2 as Region, 
                            City, 
                            Channel  as   Sub_Channel, 
                            MainChannel  as  Sub_MainChannel, 
                            NEWMainChannel1  as  NEWMainChannelforCommercialTeam, 
                            charter_noncharter  as  Charter_Noncharter,
                            Flight1 as F1, Flight2 as F2, Flight3 as F3, Flight4 as F4, 
                            Week, Pax, 
                            MainChannelBasedOnParentID,
                            sector  as  Sector, NOD, 
                            [Country_Name ] as  Country_Name , GlobalRegion  as  Global_Region,
                            [FIT/Grp_Series]  as  FIT_Grp_Series, domint  as  DOM_INT, IDType,
                            [MTD/FWD]  as  MTD_FWD
                            from canres4""")


In [ ]:
#exporting the final dataset to excel
resultcan.to_excel("cancellation_PROCESSED-06092023.xlsx",index=False)